In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
# Load the train data
trainData = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
trainData = trainData.drop(['Id'],axis = 1) # Drop the ID column as it will not help in model training
trainData.shape # to show number of (row,col)

# To see sample data
trainData.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
# Load the test data set
testData = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
testData.shape

# testData.head(5)

(1459, 80)

**Removing null values/Handling missing**

In [4]:
# Filter the columns
nullColumns = trainData.columns[trainData.isnull().sum()>500]
# print(nullColumns)
# Drop the columns
trainData = trainData.drop(nullColumns,axis=1)
# trainData = trainData.drop(['1stFlrSF','2ndFlrSF','3SsnPorch'], axis=1)
# trainData.shape

In [5]:
# Filter the columns for test data
nullColumnsTest = testData.columns[testData.isnull().sum()>500]
# print(nullColumnsTest)
# Drop the columns
testData = testData.drop(nullColumnsTest,axis=1)
# testData = testData.drop(['1stFlrSF','2ndFlrSF','3SsnPorch'], axis=1)
# testData.shape

In [6]:
# Imputing missing values
from sklearn.impute import SimpleImputer

filteredCol = trainData.columns[trainData.isnull().sum()<500]
# print(len(filteredCol))
# print(filteredCol)
imputer = SimpleImputer(strategy='most_frequent')
trainData[filteredCol] = imputer.fit_transform(trainData[filteredCol])

# print(trainData.isnull().sum())

# We could have separate categorical and numerical columns as well like as below-->
# numeric_cols = trainData[filteredCol].select_dtypes(include=['number']).columns
# categorical_cols = trainData[filteredCol].select_dtypes(exclude=['number']).columns

In [7]:
# Imputing missing values for Test Data
from sklearn.impute import SimpleImputer

filteredColTest = testData.columns[testData.isnull().sum()<500]
# print(len(filteredColTest))
# print(filteredColTest)
# print(filteredCol)
imputer = SimpleImputer(strategy='most_frequent')
testData[filteredColTest] = imputer.fit_transform(testData[filteredColTest])

# print(trainData.isnull().sum())

# We could have separate categorical and numerical columns as well like as below-->
# numeric_cols = trainData[filteredCol].select_dtypes(include=['number']).columns
# categorical_cols = trainData[filteredCol].select_dtypes(exclude=['number']).columns

**Organize train and test data**

In [8]:
# We will convert categorical value to numeric value as we will be using Random Forest Regressor
# Random Forest Regressor works with numeric values
targetFeatures = trainData.drop(['SalePrice'],axis=1)
x_train = pd.get_dummies(targetFeatures, drop_first=True)
x_test = pd.get_dummies(testData, drop_first=True)

x_train, x_test = x_train.align(x_test, join='inner', axis=1) # Ensures that train and test have same features


print(x_train.shape)
x_test.shape

(1460, 2518)


(1459, 2518)

In [9]:
from sklearn.ensemble import RandomForestRegressor

y_train = trainData['SalePrice']

model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
model.fit(x_train,y_train)

predictions = model.predict(x_test)


In [10]:
# Making the result
output = pd.DataFrame({'Id': testData.Id, 'SalePrice': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
